In [22]:
import pandas as pd
import sklearn.metrics as metrique
from pandas import Series
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Embedding, Dropout,Input, Attention, Layer, Concatenate, Permute, Dot, Multiply, Flatten
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras import backend as K, regularizers, Model, metrics
from tensorflow.keras.backend import cast
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [11]:
# x= pd.read_csv('fetureselectedcreditcard.csv', na_filter=True)
# y=pd.read_csv('Target.csv', na_filter=True)
# X_train,X_test,y_train,y_test = train_test_split(x,y, test_size=0.2,shuffle=False)

# X_train.to_csv('ur_X_train.csv',index=False)
# X_test.to_csv('ur_X_test.csv',index=False)

# y_train.to_csv('ur_y_train.csv',index=False)
# y_test.to_csv('ur_y_test.csv',index=False)


In [12]:
X_train=pd.read_csv('ur_X_train.csv', na_filter=True)
y_train=pd.read_csv('ur_y_train.csv',na_filter=True)

In [13]:
X_train.shape

(227845, 18)

In [14]:
X_test.shape

(56962, 18)

In [15]:
n=2

In [16]:
X_train0,X_train1=np.array_split(X_train,n)
y_train0,y_train1=np.array_split(y_train,n)

In [17]:
X_train0.shape,X_train1.shape

((113923, 18), (113922, 18))

In [18]:
y_train0.shape,y_train1.shape

((113923, 1), (113922, 1))

# GAN

In [23]:
def generator(z, reuse=None):
    with tf.variable_scope("gen", reuse=reuse):
        hidden1 = tf.layers.dense(inputs=z, units=128)
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1, hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1, units=64)
        hidden2 = tf.maximum(alpha*hidden2, hidden2)
        output = tf.layers.dense(hidden2, units=18, activation=tf.nn.tanh)
        return output

def discriminator(X, reuse=None):
    with tf.variable_scope("dis", reuse=reuse):
        hidden1 = tf.layers.dense(inputs=X, units=128)
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1, hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1, units=128)
        hidden2 = tf.maximum(alpha*hidden2, hidden2)
        logits = tf.layers.dense(hidden2, units=1)
        output = tf.sigmoid(logits)
        return output, logits

real_data = tf.placeholder(tf.float32, shape=[None, 18])
z = tf.placeholder(tf.float32, shape=[None, 100])

G = generator(z)

D_output_real, D_logits_real = discriminator(real_data)
D_output_fake, D_logits_fake  = discriminator(G, reuse=True)

# LOSS
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in, labels=labels_in))

D_real_loss = loss_func(D_logits_real, tf.ones_like(D_logits_real)*0.9)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_real))

D_loss = D_real_loss + D_fake_loss
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

learning_rate = 0.001
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

D_trainer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(D_loss, var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(G_loss, var_list=g_vars)

def next_batch(num, data):
    '''
    Return a total of `num` random samples. 
    '''
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data.iloc[i] for i in idx]

    return np.asarray(data_shuffle)

In [24]:
def GAN(X_train,y_train,no_of_samples_to_generate):
    X_ones = X_train[y_train.Class == 1]
    X_ones.reset_index(drop=True, inplace=True)

    batch_size = 10
    epochs = 5000
    init = tf.global_variables_initializer()
    samples = []

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(epochs):

            num_batches = len(X_ones) // batch_size
            for i in range(num_batches):

                batch_X = next_batch(batch_size, X_ones)            
                batch_z = np.random.uniform(-1,1, size=(batch_size,100))

                _ = sess.run(D_trainer, feed_dict={real_data:batch_X, z: batch_z})
                _ = sess.run(G_trainer, feed_dict={z: batch_z})         

        sample_z = np.random.uniform(-1,1,size=(no_of_samples_to_generate, 100)) # new samples will be generated
        gen_sample = sess.run(generator(z, reuse=True), feed_dict={z:sample_z})
        samples.append(gen_sample)
    X_fake = pd.DataFrame(samples[0], columns=X_train.columns)
    X_augmented_data = np.append(X_train, np.array(X_fake), axis=0)
    y_fake = np.ones(no_of_samples_to_generate)
    y_fake=y_fake.reshape(-1,1)
    y_augmented_data = np.append(y_train, y_fake, axis=0)
    return X_augmented_data,y_augmented_data

In [26]:
k0=y_train0.value_counts()[1]
k1=y_train1.value_counts()[1]
X_train0,y_train0= GAN(X_train0,y_train0,no_of_samples_to_generate=k0)
X_train1,y_train1= GAN(X_train1,y_train1,no_of_samples_to_generate=k1)

In [28]:
type(X_train0),type(y_train0)

(numpy.ndarray, numpy.ndarray)

In [29]:
X_train0.shape,X_train1.shape

((114164, 18), (114098, 18))

In [30]:
X_train0=pd.DataFrame(X_train0, columns=X_train.columns)
X_train1=pd.DataFrame(X_train1, columns=X_train.columns)
y_train0=pd.DataFrame(y_train0, columns=y_train.columns)
y_train1=pd.DataFrame(y_train1, columns=y_train.columns)

In [31]:
X_train0.to_csv('ur_X_train0.csv',index=False)
X_train1.to_csv('ur_X_train1.csv',index=False)
y_train0.to_csv('ur_y_train0.csv',index=False)
y_train1.to_csv('ur_y_train1.csv',index=False)